In [45]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [46]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [47]:
dict_by_name={}
dict_by_id={}

for i in range(len(movies.index)):
  dict_by_name[movies['title'][i]]= movies['movieId'][i]
  dict_by_id[movies['movieId'][i]]= movies['title'][i]

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [49]:
final_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
final_dataset.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,4.5,3.5,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,5.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,2.5,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.5,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,2.5,4.0,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
final_dataset.fillna(0,inplace=True)
final_dataset.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.5,3.5,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,2.5,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
final_dataset.shape

(9724, 610)

In [52]:
def encode_units(x):
    if x<=2.5:
        return 0
    if x>2.5:
        return 1

In [53]:
df_for_ar = final_dataset.T.applymap(encode_units)

In [54]:
df_for_ar.shape

(610, 9724)

In [55]:
df_for_ar.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
%pip install mlxtend --upgrade

In [57]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

In [58]:
#frequent_itemsets = apriori(df_for_ar, min_support=0.075,use_colnames=True)

In [59]:
frequent_itemsets = fpgrowth(df_for_ar, min_support=0.075, use_colnames=True)

In [60]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [61]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(296),(356),0.470492,0.516393,0.339344,0.721254,1.396715,0.096385,1.734939
1,(356),(296),0.516393,0.470492,0.339344,0.657143,1.396715,0.096385,1.544399
2,(296),(318),0.470492,0.513115,0.342623,0.728223,1.419221,0.101207,1.791488
3,(318),(296),0.513115,0.470492,0.342623,0.667732,1.419221,0.101207,1.593616
4,"(296, 356)",(318),0.339344,0.513115,0.263934,0.777778,1.515797,0.089812,2.190984
...,...,...,...,...,...,...,...,...,...
7581525,(51255),(2571),0.091803,0.421311,0.075410,0.821429,1.949694,0.036732,3.240656
7581526,(356),(51255),0.516393,0.091803,0.075410,0.146032,1.590703,0.028003,1.063502
7581527,(51255),(356),0.091803,0.516393,0.075410,0.821429,1.590703,0.028003,2.708197
7581528,(4993),(51255),0.300000,0.091803,0.075410,0.251366,2.738095,0.047869,1.213139


In [62]:
def get_recomendations(inputs,rules):
  inputs.sort()
  all_antecedents = [list(x) for x in rules['antecedents'].values]
  desired_indices=[]
  satisfied= True
  for i in range(len(all_antecedents)) :

    if len(all_antecedents[i]) != len(inputs) :
      continue

    satisfied= True

    for j in range(len(inputs)):
      if all_antecedents[i][j] != inputs[j] :
        satisfied= False

    if(satisfied):
       desired_indices.append(i)
    
  apriori_recommendations=rules.iloc[desired_indices,].sort_values(by=['lift'],ascending=False)
  apriori_recommendations.head()
  return apriori_recommendations
  

In [63]:
def get_Partial_recomendations(inputs,rules):
  inputs.sort()
  all_antecedents = [list(x) for x in rules['antecedents'].values]
  desired_indices=[]
  satisfied= True
  for i in range(len(all_antecedents)) :

    satisfied= False

    for j in range(len(all_antecedents[i])):
      for k in range(len(inputs)):
        if(all_antecedents[i][j]==inputs[k]):
          satisfied= True

    if(satisfied):
       desired_indices.append(i)
    
  apriori_recommendations=rules.iloc[desired_indices,].sort_values(by=['lift'],ascending=False)
  apriori_recommendations.head()
  return apriori_recommendations

In [64]:
def get_TopFour_Recommendations(rec_list):

  recommendation_list=[]

  n=0;
  i=0;

  while i< len(rec_list) and n<4 :

    for j in range(len(rec_list[i])):
      if(n<4):
        recommendation_list.append(rec_list[i][j])
        n=n+1
      
    i=i+1
    if(n>=4):
      break
  return recommendation_list
    

In [65]:
def getFinalRecommendations(rec_list, level, inputs, rules):

  needed= 4- len(rec_list)
  Final_list= []

  for i in range(len(rec_list)):
    Final_list.append(rec_list[i])

# Case when recommeded movies are less than four for the given sets, when we generation recomadation for subsets of the sets to get the maximum recomendations
  if(level==0):
    new_rec_list=get_Partial_recomendations(inputs,rules)
    new_rec_list = [list(x) for x in new_rec_list['consequents'].values]
    i=0;
    while i< len(new_rec_list) and len(Final_list)<4 :
      for j in range(len(new_rec_list[i])):
        if len(Final_list)<4 and Final_list.count(new_rec_list[i][j])==0:
          Final_list.append(new_rec_list[i][j])
      i=i+1
      if(len(Final_list)>=4):
        break
  
  if(len(Final_list)<4):
    Randon_List=[1]
    new_rec_list=get_Partial_recomendations(Randon_List,rules)
    new_rec_list = [list(x) for x in new_rec_list['consequents'].values]
    i=0
    while i< len(new_rec_list) and len(Final_list)<4 :
      for j in range(len(new_rec_list[i])):
        if len(Final_list)<4 and Final_list.count(new_rec_list[i][j])==0:
          Final_list.append(new_rec_list[i][j])
      i=i+1
      if(len(Final_list)>=4):
        break

  return Final_list
      

In [66]:
def convert_name_id(names):

  id_list=[]
  for i in range(len(names)):
    id_list.append(dict_by_name[names[i]])

  return id_list

In [67]:
def convert_id_name(movieids):
  id_list=[]
  for i in range(len(movieids)):
    id_list.append(dict_by_id[movieids[i]])

  return id_list

In [68]:
def make_prediction(data):
  name_inputs= data

  inputs= convert_name_id(name_inputs)

  print(inputs)
  apriori_recommendations =get_recomendations(inputs,rules)
  level=0

  if(len(apriori_recommendations)==0):
    apriori_recommendations=get_Partial_recomendations(inputs,rules)
    level=1

  if(len(apriori_recommendations)==0):
    Randon_List=[1]
    apriori_recommendations=get_Partial_recomendations(Randon_List,rules)
    level=2


  apriori_recommendations_list = [list(x) for x in apriori_recommendations['consequents'].values]

  apriori_recommendations_list

  movie_recommended=get_TopFour_Recommendations(apriori_recommendations_list)

  if(len(movie_recommended)<4):
    movie_recommended=getFinalRecommendations(movie_recommended, level, inputs, rules)

  movie_recommended= convert_id_name(movie_recommended)

  return movie_recommended


In [28]:
def print_output(output,index,df):
  if(index==1):
    df = pd.DataFrame(output, columns=['Movies'])
    df[]
  else:

  
  print(df)
  print("################################################")
  df.to_csv('my.csv', index=None)

In [69]:
dict= {'Movies':[]}
df= pd.DataFrame(dict)
with open("sample.tsv") as file:
    tsv_file= csv.reader(file, delimiter="\t")
    i=0;
    for line in tsv_file:
      print(line)
      if i !=0:
        output= make_prediction(line)
        output_string=""
        for i in range(4):
          print("{0}: {1}".format(i+1,output[i]))
          output_string=output_string+output[i]+";"
        # print_output(output_string)
        dict['Movies'].append(output_string)
        print("######################################################")
      i=i+1

df = pd.DataFrame(dict)
df.to_csv('my.csv', index=None)

['Movies']
['Toy Story (1995)']
[1]
1: Jurassic Park (1993)
2: Pulp Fiction (1994)
3: Independence Day (a.k.a. ID4) (1996)
4: Back to the Future (1985)
######################################################
['Jumanji (1995)']
[2]
1: Jurassic Park (1993)
2: Forrest Gump (1994)
3: Mrs. Doubtfire (1993)
4: Jurassic Park (1993)
######################################################
['Grumpier Old Men (1995)']
[3]
1: Lord of the Rings: The Two Towers, The (2002)
2: Star Wars: Episode VI - Return of the Jedi (1983)
3: Forrest Gump (1994)
4: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
######################################################
['Waiting to Exhale (1995)']
[4]
1: Lord of the Rings: The Two Towers, The (2002)
2: Star Wars: Episode VI - Return of the Jedi (1983)
3: Forrest Gump (1994)
4: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
######################################################
['Father of the Bride Part II 